## Subset sum

In [1]:
import random
import math

# Method to generate random sample from an array
def generate_random_solution(weights, length):
    return sorted(random.sample(weights, length))


# Method to generate all possible neighbours
def get_neighbourhood_subsets(weights, subset):
    neighbourhood = []
    
    # neighbours with one element less
    if len(subset) > 1:
        for index in range(len(subset)):
            neighbourhood.append(subset[:index] + subset[index + 1:])

    # neighbours with one element more
    # subset_set = set(subset)
    if len(subset) < len(weights):
        for item in weights:
            if item not in subset:
                neighbourhood.append(subset + [item])
    
    # replace every element in subset with one element from set
    for index in range(len(subset)):
        for set_item in weights:
            if set_item not in subset:
                neighbourhood.append(subset[:index] + [set_item] + subset[index + 1:])
    return neighbourhood


# Helper method to determine whether random_subset is better than
# normal subset. Returns True if random_subset is better, False
# otherwise
def is_better_neighbour(subset, random_subset):
    if random_subset:
        # Doubt
        return abs(sum(random_subset)) < abs(sum(subset))
    return False

# Method for performing the simulated_annealing
def simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations):
    if sum(subset) == target_sum:
        return best_subset
    if initial_temp <= final_temp:
        return best_subset
    if iterations == max_iterations:
        return best_subset
    if len(subset) == 1: # weights
        return subset
    
    #print(f"Iteration: {iterations}")
    
    neighbourhood_subsets = get_neighbourhood_subsets(weights, subset)
    # print(f"Neighbours: {neighbourhood_subsets}")
    #print(f"Total number of Neighbours: {len(neighbourhood_subsets)}")
    
    random_neighbour = random.choice(neighbourhood_subsets)
    #print(f"Random neighbour chosen: {random_neighbour} of sum: {sum(random_neighbour)}")
    
    if is_better_neighbour(subset, random_neighbour):
        #print('Better neighbour')
        subset = random_neighbour
        best_subset = subset
    else:
        r = random.uniform(0, 1)
        # S' - S / T
        val = abs(sum(random_neighbour) - sum(subset)) / float(initial_temp)
        #print(f"val: {val}")
        if r < math.exp(-val):
            subset = random_neighbour
    
    iterations += 1
    initial_temp = cooling_ratio * initial_temp
    
    #print(f"Temp: {initial_temp}")
    #print(f"Subset: {subset} and sum: {sum(subset)}")
    #print(f"Best subset: {best_subset} and sum: {sum(best_subset)}\n")
    
    return simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations)

In [2]:
def generate_list_from_weight_map(weight_map):
    final = []
    for key in weight_map.keys():
        if weight_map[key] != 0:
            final.extend([key] * weight_map[key])
    return final


In [3]:
##  function to generate a single random successor state from a given  state .Is called by the generate_1000 function

import numpy as np
def generate_one(st,packet_vec):
    for i in packet_vec:
        while True:
            loc=np.random.randint(0,6)
            if st[loc] >= i:
                st[loc] = st[loc] - i
                break
    return st

In [4]:
## generates 10000 successors and returns the unique successors calls the generate_one 10000 times

def generate_10000(state,packet_vec):
    a=[]
    for i in range(10000):
        a.append(generate_one(state.copy(),packet_vec))
    return set([tuple(i) for i in a]) ## only returns unique states 

In [5]:
##finds the best state based on the generated random states anbd the free_space, chooses the one which optimises the most 
## effective space consumed


def best_State(state_collection):
    cost=9999
    initial = [5, 28, 20, 46, 25, 35]
    for i in state_collection:
        free_space =sum(np.array([1,1,1,1,1,1]) - ((np.array(initial) - np.array(i))/np.array(initial))) 
        if cost  > ( free_space):
            locally_optimal_state = i  
        
    return locally_optimal_state

In [6]:
state =[5, 28, 20, 46, 25, 35]

In [7]:
## Main function which returns the next best state out of the 10000

def local_Search_Rooms(packet_string, current_state):
    all_generated_successors = generate_10000( current_state,packet_string)
    best_state = best_State(all_generated_successors)
    return best_state

In [8]:
# Execution starts here
weight_map = {1: 15, 2: 5, 4: 7, 5: 2, 6: 3, 7: 2, 9: 4}
# weight_map = {1: 2, 5: 2, 4: 2, 6: 2, 9: 2}
# weight_map = {7: 2, 9: 2}

total_no_of_commutes = 0

while(len(list(filter(lambda x: weight_map[x] > 0, list(weight_map.keys())))) > 0):
    weights = generate_list_from_weight_map(weight_map)

    # Generating a random subset
    subset = generate_random_solution(weights, random.randint(1, len(weights)))
    best_subset = subset

    initial_temp = 1500
    max_iterations = 500
    cooling_ratio = 0.99
    final_temp = 0.01

    #print(f"Random solution: {subset} and it's sum: {sum(subset)}")
    iterations = 1
    target_sum = 10
    final_subset = simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations)

    if(sum(final_subset) > 10):
        final_subset = simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations)

    print(f"Final solution: {final_subset} and sum: {sum(final_subset)}")
    updated_state = local_Search_Rooms(final_subset,state)
    print(f"Updated state: {updated_state}")

    for item in final_subset:
        weight_map[item] -= 1
        
    state = list(updated_state)

    print(f"Updated weight map is: {weight_map}")
    total_no_of_commutes += 1
    print(f"--commutes count : {total_no_of_commutes}")

    
print(f"Total no of commutes: {total_no_of_commutes}")


Final solution: [7, 2, 1] and sum: 10
Updated state: (5, 28, 20, 46, 22, 28)
Updated weight map is: {1: 14, 2: 4, 4: 7, 5: 2, 6: 3, 7: 1, 9: 4}
--commutes count : 1
Final solution: [5, 4, 1] and sum: 10
Updated state: (0, 23, 20, 46, 22, 28)
Updated weight map is: {1: 13, 2: 4, 4: 6, 5: 1, 6: 3, 7: 1, 9: 4}
--commutes count : 2
Final solution: [4, 5, 1] and sum: 10
Updated state: (0, 23, 20, 41, 18, 27)
Updated weight map is: {1: 12, 2: 4, 4: 5, 5: 0, 6: 3, 7: 1, 9: 4}
--commutes count : 3
Final solution: [1, 9] and sum: 10
Updated state: (0, 22, 20, 41, 9, 27)
Updated weight map is: {1: 11, 2: 4, 4: 5, 5: 0, 6: 3, 7: 1, 9: 3}
--commutes count : 4
Final solution: [2, 1, 7] and sum: 10
Updated state: (0, 22, 19, 41, 2, 25)
Updated weight map is: {1: 10, 2: 3, 4: 5, 5: 0, 6: 3, 7: 0, 9: 3}
--commutes count : 5
Final solution: [6, 4] and sum: 10
Updated state: (0, 22, 19, 31, 2, 25)
Updated weight map is: {1: 10, 2: 3, 4: 4, 5: 0, 6: 2, 7: 0, 9: 3}
--commutes count : 6
Final solution: [1,